<a href="https://colab.research.google.com/github/ramm1024/trabajotitulo/blob/main/POC_TT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai
!pip install -q pattern

In [ ]:
from openai import OpenAI
import os
from google.colab import userdata
from google.colab import drive
import pandas as pd
import time
import csv
from pattern.text.en import singularize
drive.mount('/content/drive')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KE')
client = OpenAI()
dataset_filename = "dummy12.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#df = pd.read_csv("/content/drive/MyDrive/11vo/poctestdata/dummy68.csv",sep=";")
df = pd.read_csv("/content/drive/MyDrive/11vo/poctestdata/"+dataset_filename,sep=",")

In [ ]:
#input data: str con datos unidos por ;
#output tipo_datos: lista de diccionarios con el tipo de dato encontrado y la cantidad de instancias
def send_prompt(data:str, model_choice:int)->str:
  completion = client.chat.completions.create(
    model=set_model(model_choice),
    messages=get_messages(data)
  )
  return completion.choices[0].message.content

def set_context(data:list[str])-> list[dict]:
  context = list
  return context

def set_model(model_choice:str)-> str:
  #https://openai.com/api/pricing/
  if model_choice == "3.5t":
    return "gpt-3.5-turbo"
  elif model_choice == "4":
    return "gpt-4"
  elif model_choice == "4t":
    return  "gpt-4-turbo"
  elif model_choice == "3.5t0125":
    return "gpt-3.5-turbo-instruct"
  elif model_choice == "4o":
    return "gpt-4o"
  else:
    return "gpt-3.5-turbo-0125"

def get_messages(data:str)->str:
  messages=[
                    {"role": "system", "content": "You are a data parser"},
                    {"role": "system", "content": "Your datasets come in form of a string separated by semicolons"},
                    {"role": "system", "content": "Yopu will use the dataset to infer the type of personal information that it contains"},
                    #{"role": "system", "content": "The types of data that it can contain are Personal Information, Sensitive Information, PHI, HIPAA protected information and non personal information"},
                    {"role": "system", "content": "Return only the name of the type of personal information"},
                    {"role": "user", "content": f"{data}"},
                ]
  return messages

def get_data(df):
  return ';'.join(map(str,df.tolist()))

def save_output(data):
  with open('/content/drive/MyDrive/11vo/poctestdata/out_'+dataset_filename, 'w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['data_type', 'column_name'])
    for item in data:
      writer.writerow([item,data[item]])

def get_data_types(df:pd.DataFrame)->dict[list]:
  data_types = dict()
  for column_name in df.columns:
    data_type = send_prompt(get_data(df[column_name]),'')
    print(singularize(data_type.lower()))
    sanitized_data_type = singularize(data_type.lower())
    if sanitized_data_type not in data_types:
      data_types[sanitized_data_type] = [column_name]
    else:
      data_types[sanitized_data_type].append(column_name)
    return data_types

In [ ]:
get_data_types(df)

save_output(data_types)
print(data_types)

sequence of number
{'sequence of number': ['id'], 'email': ['email_address', 'personal_email'], 'phone number': ['home_phone', 'fax_number', 'business_phone'], 'city name': ['city'], 'last name': ['last_name'], 'first name': ['first_name'], 'job title': ['job_title'], 'country': ['country_region'], 'language skill': ['notes'], 'car brand': ['company'], 'address': ['address'], 'state or region': ['state_province'], 'postal code': ['zip_postal_code']}
